In [2]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sionna.phy


import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.phy.config.seed = 42 # Set seed for reproducible results

E0000 00:00:1745203062.719844   11932 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745203062.724127   11932 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745203062.737693   11932 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745203062.737708   11932 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745203062.737711   11932 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745203062.737712   11932 computation_placer.cc:177] computation placer already registered. Please check linka

In [3]:
import sys
sys.path.append('../')
from src import SystemConfig, UeConfig, MyConfig, MySimulator, MyPUSCHConfig

In [4]:
from src.my_pusch_config import MyPUSCHConfig
from src.my_encoder import MyTBEncoder
from src.my_decoder import MyTBDecoder

from sionna.phy.mapping import BinarySource, Mapper
from sionna.phy.nr import PUSCHPilotPattern, LayerMapper, LayerDemapper, PUSCHLSChannelEstimator
from sionna.phy.ofdm import ResourceGrid, ResourceGridMapper, LinearDetector
from sionna.phy.nr.utils import generate_prng_seq
from sionna.phy.channel import AWGN, OFDMChannel, gen_single_sector_topology as gen_topology
from sionna.phy.mimo import StreamManagement
from sionna.phy.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, CDL
from sionna.phy import Block


import tensorflow as tf
import numpy as np

In [5]:
sys_cfg = SystemConfig(
    NCellId=442
)
ue_cfg = UeConfig()
my_cfg = MyConfig(sys_cfg, ue_cfg)
my_pusch_cfg = MyPUSCHConfig(my_config=my_cfg)

In [6]:
mask, pilots = my_pusch_cfg.pilot_patterns
mask.shape, pilots.shape

AttributeError: 'MyPUSCHConfig' object has no attribute 'pilot_patterns'

In [7]:
tf.config.run_functions_eagerly(False)

In [5]:
my_simulator = MySimulator(my_pusch_cfg)

[[[[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]
   ...
   [ True  True  True ...  True  True  True]
   [False False False ... False False False]
   [False False False ... False False False]]]]


In [6]:
my_simulator.Resource_Grid_Mapper._resource_grid._pilot_pattern.__dict__

{'_precision': 'single',
 '_pilots': <tf.Variable 'Variable:0' shape=(1, 1, 3888) dtype=complex64, numpy=
 array([[[-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j, -1.-1.j,  0.+0.j]]],
       dtype=complex64)>,
 '_mask': <tf.Variable 'Variable:0' shape=(1, 1, 14, 1944) dtype=int32, numpy=
 array([[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [1, 1, 1, ..., 1, 1, 1],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]], dtype=int32)>,
 '_normalize': <tf.Tensor: shape=(), dtype=bool, numpy=False>}

In [7]:
b,c,y,x = my_simulator(batch_size=2, no_scaling=1e-99, gen_prng_seq=20044, return_tx_iq=True, return_channel=False)

I0000 00:00:1744785805.467417    1654 service.cc:152] XLA service 0x16b394f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744785805.467521    1654 service.cc:160]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1744785809.744312    1654 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [22]:
my_simulator(batch_size=2, no_scaling=1e-99, gen_prng_seq=20044, return_tx_iq=False, return_channel=False);

In [23]:
my_simulator(batch_size=2, no_scaling=1e-99, gen_prng_seq=20045, return_tx_iq=False, return_channel=False);

In [24]:
%timeit my_simulator(batch_size=2, no_scaling=1e-99, gen_prng_seq=20044, return_tx_iq=False, return_channel=False);

18.6 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
tf.reduce_sum(b, axis=range(1,len(b.shape))),\
tf.reduce_sum(c, axis=range(1,len(c.shape))),\
tf.reduce_sum(y, axis=range(1,len(y.shape))),\
tf.reduce_sum(x, axis=range(1,len(x.shape)))

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2770., 2770.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([23401., 23401.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=complex64, numpy=array([ 350.75955+230.80492j , -342.63437 -45.109154j], dtype=complex64)>,
 <tf.Tensor: shape=(2,), dtype=complex64, numpy=array([12.644707-85.88226j, 12.644707-85.88226j], dtype=complex64)>)

In [9]:
your_pusch_cfg = MyPUSCHConfig(
                    my_config=MyConfig(
                        SystemConfig(NCellId=443),
                        UeConfig()
                    )
                )
my_pusch_cfg.dmrs_grid.sum(), your_pusch_cfg.dmrs_grid.sum()

((48-18j), (36+46j))

In [10]:
my_simulator.Resource_Grid_Mapper._resource_grid._pilot_pattern.__dict__

{'_precision': 'single',
 '_pilots': <tf.Variable 'Variable:0' shape=(1, 1, 3888) dtype=complex64, numpy=
 array([[[-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j, -1.-1.j,  0.+0.j]]],
       dtype=complex64)>,
 '_mask': <tf.Variable 'Variable:0' shape=(1, 1, 14, 1944) dtype=int32, numpy=
 array([[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [1, 1, 1, ..., 1, 1, 1],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]], dtype=int32)>,
 '_normalize': <tf.Tensor: shape=(), dtype=bool, numpy=False>}

In [11]:
from sionna.phy.nr import PUSCHPilotPattern
my_pilots = PUSCHPilotPattern([my_pusch_cfg]).pilots
your_pilots = PUSCHPilotPattern([your_pusch_cfg]).pilots
tf.reduce_sum(my_pilots), tf.reduce_sum(your_pilots)

(<tf.Tensor: shape=(), dtype=complex64, numpy=(48-18j)>,
 <tf.Tensor: shape=(), dtype=complex64, numpy=(36+46j)>)

In [12]:
my_simulator.update_pilots(your_pilots)

In [13]:
b,c,y,x = my_simulator(batch_size=2, no_scaling=1e-99, gen_prng_seq=20044, return_tx_iq=True, return_channel=False)
tf.reduce_sum(b, axis=range(1,len(b.shape))),\
tf.reduce_sum(c, axis=range(1,len(c.shape))),\
tf.reduce_sum(y, axis=range(1,len(y.shape))),\
tf.reduce_sum(x, axis=range(1,len(x.shape)))

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2770., 2770.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([23401., 23401.], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=complex64, numpy=array([-282.4751 +248.1613j , -224.63776+366.35632j], dtype=complex64)>,
 <tf.Tensor: shape=(2,), dtype=complex64, numpy=array([0.6447067-21.88226j, 0.6447067-21.88226j], dtype=complex64)>)

In [14]:
from src.my_scrambler import MyTB5GScrambler
scrambler = MyTB5GScrambler(n_rnti=20069,
                            n_id=442,
                            binary=True,
                            channel_type='PUSCH',
                            codeword_index=0,
                            precision='single')

In [5]:
tf.config.run_functions_eagerly(False)

In [15]:
input_bits = tf.constant([[1, 0, 1, 1, 0, 0, 1, 0, 1, 1]], dtype=tf.float32)
scrambled = scrambler(input_bits)
print(f"Input:     {input_bits.numpy()}")
print(f"Scrambled: {scrambled.numpy()}")

Input:     [[1. 0. 1. 1. 0. 0. 1. 0. 1. 1.]]
Scrambled: [[1. 0. 0. 1. 0. 0. 1. 1. 1. 0.]]


In [16]:
scrambler.__dict__

{'_precision': 'single',
 '_built': True,
 '_binary': True,
 '_check_input': True,
 '_input_shape': TensorShape([1, 10]),
 '_multi_stream': False,
 '_c_init': [657621434],
 '_sequence': <tf.Variable 'Variable:0' shape=(1, 10) dtype=float32, numpy=array([[0., 0., 1., 0., 0., 0., 0., 1., 0., 1.]], dtype=float32)>}

In [17]:
scrambler.c_init = 657621433
scrambler.__dict__

{'_precision': 'single',
 '_built': True,
 '_binary': True,
 '_check_input': True,
 '_input_shape': TensorShape([1, 10]),
 '_multi_stream': False,
 '_c_init': [657621433],
 '_sequence': <tf.Variable 'Variable:0' shape=(1, 10) dtype=float32, numpy=array([[1., 0., 1., 0., 0., 0., 0., 1., 0., 0.]], dtype=float32)>}

In [18]:
scrambled = scrambler(input_bits)
print(f"Input:     {input_bits.numpy()}")
print(f"Scrambled: {scrambled.numpy()}")

Input:     [[1. 0. 1. 1. 0. 0. 1. 0. 1. 1.]]
Scrambled: [[0. 0. 0. 1. 0. 0. 1. 1. 1. 1.]]


In [19]:
from src.my_pusch_pilot_pattern import MyPUSCHPilotPattern
patterns = MyPUSCHPilotPattern([my_pusch_cfg, your_pusch_cfg])


/workspaces/dsp/notebook/../src/my_pusch_pilot_pattern.py:143: UserWarning: DMRS port 0 used by multiple transmitters
  warnings.warn(msg)


In [35]:
tf.reduce_sum(patterns.pilots, axis=range(1,len(patterns.pilots.shape)))

<tf.Tensor: shape=(2,), dtype=complex64, numpy=array([48.-18.j, 36.+46.j], dtype=complex64)>

In [20]:
my_pusch_cfg.pilot_patterns

(array([[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]]),
 array([[-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j, -1.-1.j,  0.+0.j]]))

In [23]:
your_pusch_cfg.phy_cell_id, your_pusch_cfg.pilot_patterns

(443,
 (array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [ True,  True,  True, ...,  True,  True,  True],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]]]),
  array([[-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j,  1.+1.j,  0.+0.j]])))

In [24]:
my_pusch_cfg.phy_cell_id = 443
my_pusch_cfg.phy_cell_id, my_pusch_cfg.pilot_patterns

(443,
 (array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [ True,  True,  True, ...,  True,  True,  True],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]]]),
  array([[-1.+1.j,  0.+0.j,  1.-1.j, ...,  0.+0.j,  1.+1.j,  0.+0.j]])))

In [25]:
my_pusch_cfg.dmrs.type_a_position = 2
my_pusch_cfg.phy_cell_id, my_pusch_cfg.pilot_patterns

(443,
 (array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [ True,  True,  True, ...,  True,  True,  True],
          ...,
          [ True,  True,  True, ...,  True,  True,  True],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]]]),
  array([[ 1.-1.j,  0.+0.j, -1.+1.j, ...,  0.+0.j,  1.+1.j,  0.+0.j]])))